<h1 align='center'> 영상처리 프로그래밍 실습 7</h1>

<h6 align='right'> 2022. 4. 28. </h6>

<div class="alert alert-block alert-info">
    
- 파일 이름에서 00000000을 자신의 학번으로, name을 자신의 이름으로 수정하세요.

- 다음 줄에 자신의 이름, 학번, 학과(전공)을 적으세요.

* 이름: 김성문   &nbsp;&nbsp;          학번: 20175115    &nbsp;&nbsp;         학과(전공): 빅데이터 전공
    
</div>

- JupyterLab 문서의 최신 버전은 [JupyterLab Documentation](https://jupyterlab.readthedocs.io/en/stable/index.html#/)을  참고하라

- Markdown은 [Markdown Guide](https://www.markdownguide.org/)를 참고하라.
- [Markdown Cheat Sheet](https://www.markdownguide.org/cheat-sheet/)

* 제출 마감: 5월 4일 (수) 오후 6:00까지 최종본을 SmartLEAD제출


In [1]:
import cv2
import matplotlib.pyplot as plt

import numpy as np
print("OpenCV version", cv2.__version__)
print("NumPy version", np.__version__)

OpenCV version 4.5.2
NumPy version 1.20.3


## 지난 주 문제 1.

오늘 실습에서 사용한 적당한 영상을 하나 읽어서 OpenCV를 이용해서 영상을 표시하는 윈도우를 만들고 slidebar 인터페이스를 사용하여 감마 변환의 gamma를 0.1부터 5 사이로 조절할 수 있도록 하고, 감마가 변경될 때마다 윈도우에 감마 변환된 영상이 표시되는 프로그램을 작성하라.

In [78]:
def lut_transformation(r, lut):
    return lut[r]

def gamma_transformation_LUT(gamma):
    out = 255. ** (1. - gamma) * np.arange(256) ** gamma
    return out.round(0).clip(0,255).astype(np.uint8)

def onChange(val):
    global img2
    if val == 0:
        val = 1
    gamma = val / 100.
    lut_gamma = gamma_transformation_LUT(gamma)
        
    img_YUV = cv2.cvtColor(img, cv2.COLOR_BGR2YUV)
    img_YUV[...,0] = lut_transformation(img_YUV[...,0], lut_gamma)
    img2 = cv2.cvtColor(img_YUV, cv2.COLOR_YUV2BGR)
    cv2.imshow(winname, img2)

img = cv2.imread('grass.jpg')
img2 = img.copy()
winname = "Gamma transformation"
cv2.imshow(winname, img)

cv2.createTrackbar("gamma", winname, 100, 500, onChange)
cv2.waitKey(0)
cv2.destroyAllWindows()

## 예제 1. 

다음을 만족하는 함수 moving_average_filtering(filename)를 완성하라.

- filename으로 전달받은 파일이 존재하는지 조사해서 존재하지 않으면 오류 메시지를 콘솔에 출력하고 return 한다.
- 파일이 존재하면
  - 파일을 읽고, 원본 영상과 이동 평균 필터링된 영상을 나란히 화면에 표시한다.
  - 이동 평균 필터의 커널의 크기를 지정할 수 있는 trackBar interface를 제공한다. 단, trackBar로 지정하는 값의 범위는 1부터 51까지로 하되, 짝수일 경우 그보다 1 작은 홀수로 커널의 크기를 지정한다.
  - 지정된 커널의 크기를 이용해서 원 영상에 이동 평균 필터링이 적용된 영상을 구한다.

In [10]:
import cv2
import numpy as np
from os.path import exists

def moving_average_filtering(filename):
    def onMovingAverage(val):
        nonlocal ksize   # nonlocal 자신 보다 바깥에 있는 함수의 로컬 변수를 의미
        if val == 0:
            ksize= 1
        else:
            ksize = val // 2 *2 + 1
            # if val % 2 == 0:
            #     ksize = val -1
            # else:
            #     ksize = val

        kernel = np.ones((ksize, ksize)) / (ksize**2)   # kernel 만듬
        img_ma = cv2.filter2D(img, -1, kernel)
        img2 = np.hstack((img, img_ma))       # hstack을 이용하여 이미지 합침
        cv2.imshow(winname, img2)
    
    if exists(filename):
        img = cv2.imread(filename)
    else:
        print(f"Image file '{filename}' does not exist")
        return
        
    ksize = 1
    winname = 'Moving average filtering'
    cv2.namedWindow(winname, flags=1 | cv2.WINDOW_GUI_NORMAL)
    
    onMovingAverage(ksize)
    
    cv2.createTrackbar("kernel size", winname, ksize, 51, onMovingAverage)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

In [11]:
moving_average_filtering('grass.jpg')

## 예제 2.

예제 1에서 만든 프로그램에 다음 기능을 추가하라.

왼쪽 마우스 버튼을 클릭하면 화면에 표시되는 영상이 다음과 같이 토클되어야 한다.

(원 영상, 이동 평균 필터링된 영상) <--> (원 영상, 이동 평균 필터링된 영상, 차 영상)


In [12]:
def moving_average_filtering_2(filename):
    def onMovingAverage(val):
        nonlocal ksize   # nonlocal 자신 보다 바깥에 있는 함수의 로컬 변수를 의미
        if val == 0:
            ksize= 1
        else:
            ksize = val // 2 *2 + 1  # 홀수로 만들기

        kernel = np.ones((ksize, ksize)) / (ksize**2)   # kernel 만듬
        img_ma = cv2.filter2D(img, -1, kernel)
        
        if show_difference:
            img2 = np.hstack((img, img_ma, img-img_ma))
        else:
            img2 = np.hstack((img, img_ma))       # hstack을 이용하여 이미지 합침
        cv2.imshow(winname, img2)
        
    def onMouseClick(event, x, y, flags, params):
        nonlocal show_difference
        if event == cv2.EVENT_LBUTTONUP:
            show_difference = not show_difference
        onMovingAverage(ksize)
        
    
    if exists(filename):
        img = cv2.imread(filename)
    else:
        print(f"Image file '{filename}' does not exist")
        
    ksize = 1
    show_difference = False    # 마우스 클릭 되면 영상의 차를 보여주거나 안보여준다
    winname = 'Moving average filtering'
    cv2.namedWindow(winname, flags=1 | cv2.WINDOW_GUI_NORMAL)
    
    onMovingAverage(ksize)

    cv2.createTrackbar("kernel size", winname, ksize, 51, onMovingAverage)
    cv2.setMouseCallback(winname, onMouseClick)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

In [13]:
moving_average_filtering_2('grass.jpg')

## 문제 1. 

다음을 만족하는 함수 gamma_transformation(filename)을 완성하라.

- filename으로 전달받은 파일이 존재하는지 조사해서 존재하지 않으면 오류 메시지를 콘솔에 출력하고 return 한다.
- 파일이 존재하면
  - 파일을 읽고, 원본 영상과 gamma 변환 영상을 나란히 화면에 표시한다.
  - gamma 값을 지정할 수 있는 trackBar interface를 제공한다. 단, trackBar로 지정하는 값의 범위는 1부터 500까지로 하고, 이 값을 scaling해서 0.1과 5 사이로 조정한 값을 gamma로 한다.
  - trackBar의 초기값은 gamma=1.0에 해당하는 값으로 설정한다.
  - 지정된 gamma 값을 이용해서 원 영상을 gamma 변환한 영상을 구한다.
- 파일이 흑백 영상이면 흑백 영상을 표시하고, 컬러 영상인 경우에는 RGB를 YUV로 변환해서 Y 성분만 gamma 변환을 한 후에, 다시 RGB 영상으로 변환해서 

In [6]:
import cv2
import numpy as np
from os.path import exists

In [36]:
def gamma_transformation(filename):
    def gamma_transformation_LUT(gamma):      # gamma lookup table
        out = 255. ** (1. - gamma) * np.arange(256) ** gamma
        return out.round(0).clip(0,255).astype(np.uint8)
    
    def onMovinggam(val):
        nonlocal gamma
        if val == 0:
            gamma = 0.1
        gamma = val / 100.
        
        gam_lut = gamma_transformation_LUT(gamma)
        if len(img.shape) == 3:   # 컬러 영상이라면
            img_YUV = cv2.cvtColor(img, cv2.COLOR_BGR2YUV)                # YUV 채널로 변경
            img_YUV[...,0] = cv2.LUT(img_YUV[...,0], gam_lut)          # Y성분만 감마 변환
            img_gam = cv2.cvtColor(img_YUV, cv2.COLOR_YCrCb2BGR)        # 다시 BGR채널로 변경
        
        
        img_gam = cv2.LUT(img, gam_lut)
        img2 = np.hstack((img, img_gam))
        cv2.imshow(winname, img2)
    
    if exists(filename):
        img = cv2.imread(filename)        
    else:
        print(f"Image file '{filename}' does not exist")
        return
    
    gamma = 1
    winname = 'Gamma Transformation'

    onMovinggam(100)
    
    cv2.createTrackbar("gamma", winname, 100, 500, onMovinggam)
    
    cv2.waitKey(0)
    cv2.destroyAllWindows()

In [37]:
gamma_transformation('grass.jpg')

## 문제 1.
예제 2의 프로그램을 다음과 같이 변경하라.

- 이동 평균 필터링을 가우시안 필터링으로 변경하라.
- 가우시안 커널의 표준편차인 sigmaX와 sigmaY를 개별적으로 조정할 수 있는 trackBar interface를 2 개 제공한다. 단, 표준 편차의 최댓값은 10.0이 되도록 한다.


In [1]:
import cv2
import numpy as np
from os.path import exists

In [7]:
def Gaussian_filtering(filename):
    def onMovingAverage(val):
        nonlocal ksize, sigmaX, sigmaY   # nonlocal 자신 보다 바깥에 있는 함수의 로컬 변수를 의미
        if val == 0:
            ksize= 1
        else:
            ksize = val // 2 *2 + 1  # 홀수로 만들기

        img_ma = cv2.GaussianBlur(img, (ksize, ksize), sigmaX, sigmaY)
        
        if show_difference:
            img2 = np.hstack((img, img_ma, img-img_ma))
        else:
            img2 = np.hstack((img, img_ma))       # hstack을 이용하여 이미지 합침
        cv2.imshow(winname, img2)
        
    def onMovingSigmaX(val):
        nonlocal sigmaX, sigmaY, ksize
        if val == 0:
            sigmaX = 1
        else:
            sigmaX = val/10
        
        img_ma = cv2.GaussianBlur(img, (ksize, ksize), sigmaX, sigmaY)
        
        if show_difference:
            img2 = np.hstack((img, img_ma, img-img_ma))
        else:
            img2 = np.hstack((img, img_ma))       # hstack을 이용하여 이미지 합침
        cv2.imshow(winname, img2)
        
        
    def onMovingSigmaY(val):
        nonlocal sigmaX, sigmaY, ksize
        if val == 0:
            sigmaY = 1
        else:
            sigmaY = val/10
        
        img_ma = cv2.GaussianBlur(img, (ksize, ksize), sigmaX, sigmaY)
        
        if show_difference:
            img2 = np.hstack((img, img_ma, img-img_ma))
        else:
            img2 = np.hstack((img, img_ma))       # hstack을 이용하여 이미지 합침
        cv2.imshow(winname, img2)
        
    def onMouseClick(event, x, y, flags, params):
        nonlocal show_difference
        if event == cv2.EVENT_LBUTTONUP:
            show_difference = not show_difference
        onMovingAverage(ksize)
        
    
    if exists(filename):
        img = cv2.imread(filename)
    else:
        print(f"Image file '{filename}' does not exist")
        
    ksize = 1
    show_difference = False    # 마우스 클릭 되면 영상의 차를 보여주거나 안보여준다
    winname = 'Moving average filtering'
    cv2.namedWindow(winname, flags=1 | cv2.WINDOW_GUI_NORMAL)
    
    sigmaX = 1
    sigmaY = 1

    onMovingAverage(ksize)
    
    cv2.createTrackbar("kernel size", winname, ksize, 51, onMovingAverage)
    cv2.createTrackbar("sigmaX", winname, 1, 100, onMovingSigmaX)
    cv2.createTrackbar("sigmaY", winname, 1, 100, onMovingSigmaY)
    cv2.setMouseCallback(winname, onMouseClick)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

In [8]:
Gaussian_filtering('grass.jpg')